In [10]:
import pandas as pd
import requests


In [2]:
df = pd.read_csv('amazon_products.csv')

In [5]:
url = df['url'] 

In [7]:
url[1]

'https://www.amazon.com/ROPVACNIC-Automatic-Self-Charging-Scheduled-Cleaning/dp/B0DFGWWYJF'

In [8]:
data = url
df = pd.DataFrame(data)

# Hàm để trích xuất ASIN từ URL
def extract_asin(url):
    try:
        # Tách chuỗi để tìm phần có "dp/" và lấy ASIN sau đó
        return url.split("/dp/")[1].split("/")[0]
    except IndexError:
        return None

# Áp dụng hàm để trích xuất ASIN từ cột URL
df['ASIN'] = df['url'].apply(extract_asin)

# Hiển thị DataFrame với cột ASIN mới
print(df)


                                                  url        ASIN
0   https://www.amazon.com/iRobot-Roomba-EVO-Robot...  B0CNL2KCVR
1   https://www.amazon.com/ROPVACNIC-Automatic-Sel...  B0DFGWWYJF
2   https://www.amazon.com/Shark-AV753-Connected-R...  B08C6XGP4F
3   https://www.amazon.com/iRobot-Roomba-Essential...  B0CNKX4429
4   https://www.amazon.com/eufy-Super-Thin-Super-S...  B07R295MLS
..                                                ...         ...
75  https://www.amazon.com/Shark-Ultra-Robot-Vacuu...  B0BPC54599
76  https://www.amazon.com/Lefant-M310-Cleaner-450...  B0CS8X8K85
77  https://www.amazon.com/Robot-Vacuum-Mop-Combo-...  B0CB3ZPD4K
78  https://www.amazon.com/eufy-BoostIQ-Powerful-S...  B081J646XW
79  https://www.amazon.com/Shark-AV911S-Self-Empty...  B08WJMN2YG

[80 rows x 2 columns]


In [9]:
output_path = 'amazon_product_asin.csv'
df['ASIN'].to_csv(output_path, index=False)

In [ ]:


# Define the URL of the GraphQL endpoint
url = 'https://graphql.canopyapi.co/'

# Define the GraphQL query
query = """
query amazonProduct($asin: String!) {
  amazonProduct(input: {asin: $asin}) {
     rating 
              ratingsTotal
              ratingsBreakdown{
                oneStarRatingsCount
                twoStarRatingsCount
                threeStarRatingsCount
                fourStarRatingsCount
                fiveStarRatingsCount
              }
    reviewsTotal
    reviewsPaginated(input: {page: 1}) {
      reviews {
        title
        body
        rating
        helpfulVotes
        reviewer {
          name
        }
      }
    }
    title
    subtitle
    brand
    asin
    url
    price {
      symbol
      value
      currency
      display
    }
    categories {
      name
    }
    countryOfOrigin
    featureBullets
    technicalSpecifications {
      name
      value
    }
    salesEstimate {
      weeklyUnitSales
      monthlyUnitSales
      annualUnitSales
    }
    stockEstimate {
      offerId
      inStock
      stockLevel
      availabilityMessage
    }
  }
}
"""

headers = {
    'Content-Type': 'application/json',
    'API-KEY': 'cc52e86c-07ae-4b3a-a14c-d2ed2c93edc4',
}

variables = {
  'asin': 'B07R295MLS',
}

# Define the request payload
payload = {
    'query': query,
    'variables': variables
}

# Send the POST request to the GraphQL endpoint
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Query failed to run with a {response.status_code} status code.")
    print(f"Response: {response.text}")

In [11]:
import requests

# Đọc file CSV chứa các ASIN
asin_file = 'amazon_product_asin.csv'
df = pd.read_csv(asin_file)

# Lấy danh sách các ASIN
asin_list = df['ASIN'].tolist()

# Tạo query GraphQL động
def create_graphql_query(asin_list):
    query = "query multipleAmazonProducts {\n"
    
    for i, asin in enumerate(asin_list):
        query += f"""
        product{i+1}: amazonProduct(input: {{ asinLookup: {{ asin: "{asin}" }} }}) {{
            reviewsTotal
            reviewsPaginated(input: {{ page: 1 }}) {{
              reviews {{
                title
                body
                rating
                helpfulVotes
                reviewer {{
                  name
                }}
              }}
            }}
        }}\n"""
    
    query += "}"
    return query

# Tạo query từ danh sách ASIN
graphql_query = create_graphql_query(asin_list)

# Define the URL of the GraphQL endpoint
url = 'https://graphql.canopyapi.co/'

# Define the headers
headers = {
    'Content-Type': 'application/json',
    'API-KEY': 'cc52e86c-07ae-4b3a-a14c-d2ed2c93edc4',  # Thay thế bằng API Key của bạn
}

# Define the request payload
payload = {
    'query': graphql_query
}

# Gửi yêu cầu POST đến GraphQL endpoint
response = requests.post(url, json=payload, headers=headers)

# Kiểm tra phản hồi
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Query failed with status code {response.status_code}")
    print(f"Response: {response.text}")


{'data': {'product1': {'reviewsPaginated': {'reviews': [{'title': 'The iRobot I7+; the Good and Bad', 'body': 'Well, it\'s been about 18 months since I bought the iRobot i7+ and so far things are going fine; no problems with it so far. I did do a good cleaning recently and, after looking at instructions on various YouTube site I found how easy it was to clean around wheels, clean single wheel in front of iRobot, to remove strings etc. I started off without researching and was struggling but after finding out how to really do the job it was a piece of cake. The iRobot designers really too all this cleaning into consideration when designing the product.  So far, maintains 5 star rating.  The following is a summary of my experience with my new iRobot I7+  I bought this vacuum to replace my Neato Connected which was very good at cleaning but left a lot as far as reliability; in less than 5 years I had to have it repaired 3 times at a cost of almost $400. This for a relatively expensive rob